<a href="https://colab.research.google.com/github/PranavDixit2/LungCancer/blob/main/Programing_Assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras import backend as K

In [16]:
def conv_block_2conv(x, filters):
    """
    Perform two consecutive 3x3 convolutions with 'valid' padding,
    each followed by ReLU activation.
    Dimensions will shrink by 2 on each side per convolution (total of 4 per conv_block).
    """
    x = layers.Conv2D(filters, kernel_size=3, activation='relu', padding='valid')(x)
    x = layers.Conv2D(filters, kernel_size=3, activation='relu', padding='valid')(x)
    return x

In [17]:
def crop_and_concat(skip_connection, upsampled):
    # Get shapes of skip_connection and upsampled
    sc_shape = K.int_shape(skip_connection)
    up_shape = K.int_shape(upsampled)

    # Calculate the height and width differences
    height_diff = sc_shape[1] - up_shape[1]
    width_diff = sc_shape[2] - up_shape[2]

    # Crop the skip connection tensor
    cropped_skip = skip_connection[:,
                                   height_diff // 2 : sc_shape[1] - (height_diff - height_diff // 2),
                                   width_diff  // 2 : sc_shape[2] - (width_diff  - width_diff  // 2),
                                   :]
    return layers.Concatenate(axis=-1)([cropped_skip, upsampled])

In [18]:
def build_unet_ronneberger(input_shape=(572, 572, 1), num_classes=2):
    """
    Build the U-Net model as described in Ronneberger et al. (2015)
    with valid padding to achieve a final output of 388x388 for 572x572 input.
    """

    #########################################################
    ## Your code here:
    ## Build an input layer using the layers module with shape of input_shape:
    inputs = layers.Input(shape=input_shape)
    #########################################################


    # ------------------- Encoder (Contracting Path) -------------------
    # Block 1
    conv1 = conv_block_2conv(inputs, 64)              # -> 568 x 568
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)  # -> 284 x 284

    # Block 2
    conv2 = conv_block_2conv(pool1, 128)             # -> 280 x 280
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)  # -> 140 x 140

    # Block 3
    #########################################################
    ## Your code here:
    ## Use the conv_block_2conv function to create a convolutional block with 256 filters, and then create a 2D maxpooling layer using the layer module with pool size of (2, 2):
    conv3 = conv_block_2conv(pool2, 256)             # -> 136 x 136
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)  # -> 68 x 68
    #########################################################

    # Block 4
    #########################################################
    ## Your code here:
    ## Use the conv_block_2conv function to create a convolutional block with 512 filters, and then create a 2D maxpooling layer using the layer module to halve the image size in the x and y directions:
    conv4 = conv_block_2conv(pool3, 512)             # -> 64 x 64
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)  # -> 32 x 32
    #########################################################

    # Bottom (Block 5)
    conv5 = conv_block_2conv(pool4, 1024)            # -> 28 x 28

    # ------------------- Decoder (Expanding Path) -------------------
    # Up Block 4
    up4 = layers.Conv2DTranspose(512, kernel_size=2, strides=2, padding='valid')(conv5)  # -> 56 x 56
    merge4 = crop_and_concat(conv4, up4)             # crop conv4 (64 x 64) -> (56 x 56), concat
    conv6 = conv_block_2conv(merge4, 512)            # -> 52 x 52

    # Up Block 3
    up3 = layers.Conv2DTranspose(256, kernel_size=2, strides=2, padding='valid')(conv6)  # -> 104 x 104
    merge3 = crop_and_concat(conv3, up3)             # crop conv3 (136 x 136) -> (104 x 104)
    conv7 = conv_block_2conv(merge3, 256)            # -> 100 x 100

    # Up Block 2
    #########################################################
    ## Your code here:
    ## Create a Conv2DTranspose layer with 128 filters using the layer module to increase the image dimension to 200x200:
    ## Hint: use kernel size of 2, strides of 2, and padding of 'valid'
    up2 = layers.Conv2DTranspose(128, kernel_size=2, strides=2, padding='valid')(conv7)  # -> 200 x 200
    merge2 = crop_and_concat(conv2, up2)             # crop conv2 (280 x 280) -> (200 x 200)
    conv8 = conv_block_2conv(merge2, 128)            # -> 196 x 196
    #########################################################

    # Up Block 1
    #########################################################
    ## Your code here:
    ## Use the crop_and_concat function to crop the image and incorporate skip connections with the image with the dimension of 568 x 568:
    ## Hint: Which feature map earlier in your network has the size of 568 x 568?
    up1 = layers.Conv2DTranspose(64, kernel_size=2, strides=2, padding='valid')(conv8)   # -> 392 x 392
    merge1 = crop_and_concat(conv1, up1)             #                           # crop feature map of size (568 x 568) -> (392 x 392)
    conv9 = conv_block_2conv(merge1, 64)             # -> 388 x 388
    #########################################################

    # Output layer: 1x1 convolution to map features to num_classes (e.g., 2)
    outputs = layers.Conv2D(num_classes, kernel_size=1, activation='softmax')(conv9)

    model = Model(inputs=inputs, outputs=outputs, name="UNet_Ronneberger2015")
    return model

In [19]:
# Build and check the model summary
if __name__ == "__main__":
    unet_model = build_unet_ronneberger()
    unet_model.summary()

Model: "UNet_Ronneberger2015"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 572, 572, 1)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_40 (Conv2D)        │ (None, 570, 570, 64)   │            640 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_41 (Conv2D)        │ (None, 568, 568, 64)   │         36,928 │ conv2d_40[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_16          │ (None, 284, 284, 64)   │              0 │ conv2d_41[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_42 (Conv2D)        │ (None, 282, 282, 128)  │         73,856 │ max_pooling2d_16[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_43 (Conv2D)        │ (None, 280, 280, 128)  │        147,584 │ conv2d_42[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_17          │ (None, 140, 140, 128)  │              0 │ conv2d_43[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_44 (Conv2D)        │ (None, 138, 138, 256)  │        295,168 │ max_pooling2d_17[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_45 (Conv2D)        │ (None, 136, 136, 256)  │        590,080 │ conv2d_44[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_18          │ (None, 68, 68, 256)    │              0 │ conv2d_45[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_46 (Conv2D)        │ (None, 66, 66, 512)    │      1,180,160 │ max_pooling2d_18[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_47 (Conv2D)        │ (None, 64, 64, 512)    │      2,359,808 │ conv2d_46[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_19          │ (None, 32, 32, 512)    │              0 │ conv2d_47[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_48 (Conv2D)        │ (None, 30, 30, 1024)   │      4,719,616 │ max_pooling2d_19[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_49 (Conv2D)        │ (None, 28, 28, 1024)   │      9,438,208 │ conv2d_48[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item (GetItem)        │ (None, 56, 56, 512)    │              0 │ conv2d_47[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_4        │ (None, 56, 56, 512)    │      2,097,664 │ conv2d_49[0][0]        │
│ (Conv2DTranspose)    

 Total params: 31,030,658 (118.37 MB)

 Trainable params: 31,030,658 (118.37 MB)

 Non-trainable params: 0 (0.00 B)